In [7]:
import os
import random
import itertools
import re

# 基本的なライブラリ
import numpy as np
import pandas as pd
from scipy import stats

# 描画ライブラリ
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
import pydotplus
from IPython.display import Image, HTML
from six import StringIO
from ipywidgets import interact, FloatSlider
from dtreeviz.trees import *

# データセット分割
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    ShuffleSplit,
    StratifiedShuffleSplit,
)

# 補完
from sklearn.experimental import (
    enable_iterative_imputer,
)  # IterativeImputerをimportするために必要
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

# エンコード
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder

# サンプリング
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# 特徴量選択
from sklearn.feature_selection import (
    VarianceThreshold,
    GenericUnivariateSelect,
    f_classif,
    mutual_info_classif,
    chi2,
)
from boruta import BorutaPy
from libraries.mrmr import mrmr

# 学習中
import optuna
from tqdm import tqdm
from sklearn.model_selection import learning_curve, cross_validate, cross_val_score

# 評価指標
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# models
from lightgbm import LGBMClassifier
import xgboost as xgb

# XAI
import shap

import warnings


# config python file
import config

SEED = config.SEED
INDEX_MICROARRAY = config.INDEX_MICROARRAY
import functions

functions.fix_seed(SEED)


# 最大表示列数の指定（ここでは50列を指定）N
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)

%matplotlib inline

In [8]:
df_recurrenced = pd.read_pickle(
    config.INTERIM_PREPROCESSED_RECURRENCE_CROSS_DIR + "/df_recurrenced.pkl"
)

# 最適化

分類器を学習させ、パラメータのチューニングを行い、高い予測精度を目指す。

## Random Forest

In [ ]:
f1_score()

In [34]:
def objective(trial):
    # ランダムフォレストのパラメータチューニング
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 1000),
        "max_depth": trial.suggest_int("max_depth", 2, 50, log=True),
        "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 2, 100),
        "class_weight": "balanced",
        "random_state": SEED,
    }

    clf = RandomForestClassifier(**params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(
        clf, X_train, y_train, n_jobs=-1, cv=10, scoring=make_scorer(f1_score)
    )
    return score.mean()


# サブグループ削除（一時的に）
subgroup_columns = [
    "CLAUDIN_SUBTYPE",
    # "NPI_CAT",
    # "TUMOR_CAT",
    # "LYMPH_CAT",
]

methods = [
    # "variance_threshold",
    "mrmr"
]

for method in methods:  # 特徴選択手法の適用
    for subgroup_column in subgroup_columns:  # 各サブグループへの適用
        print("-----" * 10)
        print("subgroup column: ", subgroup_column)
        for subgroup in df_recurrenced[subgroup_column].unique():  # サブグループ毎への適用
            print("subgroup: ", subgroup)
            file_path = "./{0}/{1}/{2}".format(
                config.INTERIM_PREPROCESSED_RECURRENCE_CROSS_DIR,
                subgroup_column,
                method,
            )
            file_name = subgroup
            functions.make_dir(file_path)
            try:
                list_train, list_val, _, _ = functions.read_preprocessed_df(
                    file_path, file_name
                )
            except:
                print("train data does not exist")
                continue
            X_train, y_train = list_train[0], list_train[1].map(lambda x: int(x))
            X_val, y_val = list_val[0], list_val[1].map(lambda x: int(x))
            optuna.logging.set_verbosity(optuna.logging.WARNING)
            study = optuna.create_study(
                direction="maximize", sampler=optuna.samplers.RandomSampler(seed=SEED)
            )
            study.optimize(objective, n_trials=100, show_progress_bar=True)
            print(study.best_params)
            print(study.best_value)
            # params save
            file_path = "./{0}/{1}/{2}/{3}".format(
                config.INTERIM_TUNING_RECURRENCE_CROSS_DIR,
                subgroup_column,
                method,
                "RandomForest",
            )
            functions.make_dir(file_path)
            file_name_paramas = "/params_{0}.pkl".format(subgroup)
            file_name_value = "/value_{0}.pkl".format(subgroup)
            functions.pickle_dump(study.best_params, file_path + file_name_paramas)
            functions.pickle_dump(study.best_value, file_path + file_name_value)

--------------------------------------------------
subgroup column:  CLAUDIN_SUBTYPE
subgroup:  LumB


  0%|          | 0/100 [00:00<?, ?it/s]

{'n_estimators': 595, 'max_depth': 2, 'max_leaf_nodes': 82}
0.705415417915418
subgroup:  Her2


  0%|          | 0/100 [00:00<?, ?it/s]

{'n_estimators': 193, 'max_depth': 2, 'max_leaf_nodes': 23}
0.6240476190476191
subgroup:  LumA


  0%|          | 0/100 [00:00<?, ?it/s]

{'n_estimators': 818, 'max_depth': 4, 'max_leaf_nodes': 44}
0.8183660965946602
subgroup:  claudin-low
pkl file does not exist
train data does not exist
subgroup:  Basal
pkl file does not exist
train data does not exist
subgroup:  Normal
pkl file does not exist
train data does not exist
subgroup:  NC
pkl file does not exist
train data does not exist


### LogisticRegression

In [33]:
def objective(trial):
    params = {
        "penalty": trial.suggest_categorical("penalty", ["l1", "l2"]),
        "C": trial.suggest_float("C", 1e-5, 1e5),
        "max_iter": 1000,
        "class_weight": "balanced",
        "random_state": SEED,
    }

    clf = LogisticRegression(**params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(
        clf, X_train, y_train, n_jobs=-1, cv=10, scoring=make_scorer(f1_score)
    )
    return score.mean()


# サブグループ削除（一時的に）
subgroup_columns = [
    "CLAUDIN_SUBTYPE",
    # "NPI_CAT",
    # "TUMOR_CAT",
    # "LYMPH_CAT",
]

methods = [
    # "variance_threshold",
    "mrmr"
]

for method in methods:  # 特徴選択手法の適用
    for subgroup_column in subgroup_columns:  # 各サブグループへの適用
        print("-----" * 10)
        print("subgroup column: ", subgroup_column)
        for subgroup in df_recurrenced[subgroup_column].unique():  # サブグループ毎への適用
            print("subgroup: ", subgroup)
            file_path = "./{0}/{1}/{2}".format(
                config.INTERIM_PREPROCESSED_RECURRENCE_CROSS_DIR,
                subgroup_column,
                method,
            )
            file_name = subgroup
            functions.make_dir(file_path)
            try:
                list_train, list_val, _, _ = functions.read_preprocessed_df(
                    file_path, file_name
                )
            except:
                print("train data does not exist")
                continue
            X_train, y_train = list_train[0], list_train[1].map(lambda x: int(x))
            X_val, y_val = list_val[0], list_val[1].map(lambda x: int(x))
            optuna.logging.set_verbosity(optuna.logging.WARNING)
            study = optuna.create_study(
                direction="maximize", sampler=optuna.samplers.RandomSampler(seed=SEED)
            )
            study.optimize(objective, n_trials=100, show_progress_bar=True)
            print(study.best_params)
            print(study.best_value)
            # params save
            file_path = "./{0}/{1}/{2}/{3}".format(
                config.INTERIM_TUNING_RECURRENCE_CROSS_DIR,
                subgroup_column,
                method,
                "LogisticRegression",
            )
            functions.make_dir(file_path)
            file_name_paramas = "/params_{0}.pkl".format(subgroup)
            file_name_value = "/value_{0}.pkl".format(subgroup)
            pickle_dump(study.best_params, file_path + file_name_paramas)
            pickle_dump(study.best_value, file_path + file_name_value)

--------------------------------------------------
subgroup column:  CLAUDIN_SUBTYPE
subgroup:  LumB


  0%|          | 0/100 [00:00<?, ?it/s]

[W 2022-10-12 10:16:38,269] Trial 0 failed, because the objective function returned nan.
[W 2022-10-12 10:16:38,845] Trial 1 failed, because the objective function returned nan.
[W 2022-10-12 10:16:39,232] Trial 4 failed, because the objective function returned nan.
[W 2022-10-12 10:16:39,272] Trial 5 failed, because the objective function returned nan.
[W 2022-10-12 10:16:39,326] Trial 6 failed, because the objective function returned nan.
[W 2022-10-12 10:16:39,382] Trial 7 failed, because the objective function returned nan.
[W 2022-10-12 10:16:40,273] Trial 11 failed, because the objective function returned nan.
[W 2022-10-12 10:16:40,286] Trial 12 failed, because the objective function returned nan.
[W 2022-10-12 10:16:40,474] Trial 14 failed, because the objective function returned nan.
[W 2022-10-12 10:16:40,866] Trial 17 failed, because the objective function returned nan.
[W 2022-10-12 10:16:40,882] Trial 18 failed, because the objective function returned nan.
[W 2022-10-12 10

  0%|          | 0/100 [00:00<?, ?it/s]

[W 2022-10-12 10:16:49,327] Trial 0 failed, because the objective function returned nan.
[W 2022-10-12 10:16:49,354] Trial 1 failed, because the objective function returned nan.
[W 2022-10-12 10:16:50,359] Trial 4 failed, because the objective function returned nan.
[W 2022-10-12 10:16:50,374] Trial 5 failed, because the objective function returned nan.
[W 2022-10-12 10:16:50,391] Trial 6 failed, because the objective function returned nan.
[W 2022-10-12 10:16:50,410] Trial 7 failed, because the objective function returned nan.
[W 2022-10-12 10:16:50,813] Trial 11 failed, because the objective function returned nan.
[W 2022-10-12 10:16:50,828] Trial 12 failed, because the objective function returned nan.
[W 2022-10-12 10:16:50,994] Trial 14 failed, because the objective function returned nan.
[W 2022-10-12 10:16:51,319] Trial 17 failed, because the objective function returned nan.
[W 2022-10-12 10:16:51,339] Trial 18 failed, because the objective function returned nan.
[W 2022-10-12 10

  0%|          | 0/100 [00:00<?, ?it/s]

[W 2022-10-12 10:16:58,048] Trial 0 failed, because the objective function returned nan.
[W 2022-10-12 10:16:58,064] Trial 1 failed, because the objective function returned nan.
[W 2022-10-12 10:16:58,303] Trial 4 failed, because the objective function returned nan.
[W 2022-10-12 10:16:58,321] Trial 5 failed, because the objective function returned nan.
[W 2022-10-12 10:16:58,334] Trial 6 failed, because the objective function returned nan.
[W 2022-10-12 10:16:58,349] Trial 7 failed, because the objective function returned nan.
[W 2022-10-12 10:16:58,723] Trial 11 failed, because the objective function returned nan.
[W 2022-10-12 10:16:58,734] Trial 12 failed, because the objective function returned nan.
[W 2022-10-12 10:16:58,868] Trial 14 failed, because the objective function returned nan.
[W 2022-10-12 10:16:59,127] Trial 17 failed, because the objective function returned nan.
[W 2022-10-12 10:16:59,146] Trial 18 failed, because the objective function returned nan.
[W 2022-10-12 10

In [9]:
def objective(trial):
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "max_bin": trial.suggest_int("max_bin", 100, 300),
        "num_leaves": trial.suggest_int("num_leaves", 20, 50),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
        "n_estimators": trial.suggest_int("n_estimators", 10, 1000),
        "class_weight": "balanced",
        "random_state": SEED,
    }
    clf = LGBMClassifier(boosting_type="gbdt", **params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=10)
    return score.mean()


dict_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"] = dict()
dict_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"] = dict()

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, show_progress_bar=True)
dict_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"][
    config.THRESHOLD_YEAR
] = study.best_trial.values
dict_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"][
    config.THRESHOLD_YEAR
] = study.best_trial.params

[I 2022-09-20 14:00:24,734] A new study created in memory with name: no-name-fe85ca54-b769-486f-9063-10ac11b643fa


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2022-09-20 14:00:26,602] Trial 0 finished with value: 0.7352044907778669 and parameters: {'lambda_l1': 5.2650661682626334e-08, 'lambda_l2': 2.676685804656544e-07, 'max_bin': 205, 'num_leaves': 39, 'learning_rate': 0.020500314953950424, 'n_estimators': 410}. Best is trial 0 with value: 0.7352044907778669.
[I 2022-09-20 14:00:26,836] Trial 1 finished with value: 0.6462443197006149 and parameters: {'lambda_l1': 1.9611027454558388e-05, 'lambda_l2': 0.009859684973036029, 'max_bin': 156, 'num_leaves': 26, 'learning_rate': 0.0018371964532022608, 'n_estimators': 42}. Best is trial 0 with value: 0.7352044907778669.
[I 2022-09-20 14:00:27,492] Trial 2 finished with value: 0.7376369954557604 and parameters: {'lambda_l1': 0.04491901149457346, 'lambda_l2': 0.0011661959910222177, 'max_bin': 183, 'num_leaves': 22, 'learning_rate': 0.05163262200039377, 'n_estimators': 189}. Best is trial 2 with value: 0.7376369954557604.
[I 2022-09-20 14:00:28,729] Trial 3 finished with value: 0.7156375300721732 an

In [11]:
make_dir(config.INTERIM_TUNING_PROGNOSIS_CROSS_DIR)
pickle_dump(
    dict_optim_values,
    config.INTERIM_TUNING_PROGNOSIS_CROSS_DIR + "/microarray_optim_values.pickle",
)
pickle_dump(
    dict_optim_params,
    config.INTERIM_TUNING_PROGNOSIS_CROSS_DIR + "/microarray_optim_params.pickle",
)

In [14]:
def objective(trial):
    warnings.simplefilter("ignore")
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "max_bin": trial.suggest_int("max_bin", 100, 300),
        "num_leaves": trial.suggest_int("num_leaves", 20, 50),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
        "n_estimators": trial.suggest_int("n_estimators", 10, 1000),
        "class_weight": "balanced",
        "random_state": SEED,
    }
    clf = LGBMClassifier(boosting_type="gbdt", **params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(clf, X_train_subtype, y_train_subtype, n_jobs=-1, cv=10)
    return score.mean()


optuna.logging.disable_default_handler()
warnings.simplefilter("ignore")
subtypes = [
    "claudin-low",
    "LumA",
    "LumB",
    "Her2",
    "Normal",
    "Basal",
]
dict_subtype_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"] = dict()
dict_subtype_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"] = dict()


for subtype in tqdm(subtypes):
    print("----------" * 10)
    print(subtype)
    dict_subtype_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"][
        subtype
    ] = dict()
    dict_subtype_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"][
        subtype
    ] = dict()
    prognosis_Xlabel = "X{0}_{1}".format(config.THRESHOLD_YEAR, subtype)
    prognosis_ylabel = "y{0}_{1}".format(config.THRESHOLD_YEAR, subtype)
    X_train_subtype = X_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]][
        "claudin_subtype"
    ]["mrmr"]["train"][prognosis_Xlabel]
    y_train_subtype = y_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]][
        "claudin_subtype"
    ]["mrmr"]["train"][prognosis_ylabel]
    assert (
        X_train_subtype.shape[0] == y_train_subtype.shape[0]
    ), "train size is incorrect"

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, show_progress_bar=True)
    dict_subtype_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"][
        subtype
    ][config.THRESHOLD_YEAR] = study.best_trial.values
    dict_subtype_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"][
        subtype
    ][config.THRESHOLD_YEAR] = study.best_trial.params

=====15=====15=====15=====15=====15=====15=====15=====15=====15=====15


  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

----------------------------------------------------------------------------------------------------
claudin-low


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|                                                                                            | 0/6 [00:01<?, ?it/s]

...

[I 2022-09-20 14:10:16,067] Trial 93 finished with value: 0.7318181818181818 and parameters: {'lambda_l1': 0.2142546289774351, 'lambda_l2': 1.9211362812119187e-06, 'max_bin': 140, 'num_leaves': 36, 'learning_rate': 0.008086770329420503, 'n_estimators': 225}. Best is trial 71 with value: 0.78.
[I 2022-09-20 14:10:16,129] Trial 94 finished with value: 0.7509090909090909 and parameters: {'lambda_l1': 0.14637004752311344, 'lambda_l2': 6.750387512001505e-07, 'max_bin': 176, 'num_leaves': 46, 'learning_rate': 0.012472317626717898, 'n_estimators': 227}. Best is trial 71 with value: 0.78.
[LightGBM] [Warning] lambda_l2 is set=8.058127235244045, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.058127235244045
[LightGBM] [Warning] lambda_l1 is set=0.022974998584609687, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.022974998584609687
[LightGBM] [Warning] lambda_l2 is set=0.0002624961743764266, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0002624961743764266
[Li

 83%|██████████████████████████████████████████████████████████████████████              | 5/6 [01:28<00:14, 14.09s/it]

[I 2022-09-20 14:10:16,268] Trial 96 finished with value: 0.7418181818181818 and parameters: {'lambda_l1': 0.0008913739998469707, 'lambda_l2': 4.288222326930754e-05, 'max_bin': 220, 'num_leaves': 48, 'learning_rate': 0.013791227667556595, 'n_estimators': 419}. Best is trial 71 with value: 0.78.
[LightGBM] [Warning] lambda_l1 is set=0.001301007448502127, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.001301007448502127
[LightGBM] [Warning] lambda_l2 is set=2.256268008729143e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.256268008729143e-07
[LightGBM] [Warning] lambda_l1 is set=8.474930225544429e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.474930225544429e-05
[LightGBM] [Warning] lambda_l2 is set=2.997310022112025e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.997310022112025e-07
[LightGBM] [Warning] lambda_l1 is set=0.001070373191532369, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.001070373191532369
[LightGBM] 

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:28<00:00, 14.70s/it]

[I 2022-09-20 14:10:16,472] Trial 99 finished with value: 0.6390909090909089 and parameters: {'lambda_l1': 5.917311103814676e-07, 'lambda_l2': 5.400781951264527e-06, 'max_bin': 156, 'num_leaves': 49, 'learning_rate': 0.0010760922450708738, 'n_estimators': 163}. Best is trial 71 with value: 0.78.
